# Unwetter Simulator

In [1]:
import os
os.chdir('..')

f'Working directory: {os.getcwd()}'

'Working directory: /home/jhoeke/git/p3nny/unwetter'

In [2]:
from unwetter import db, map
from datetime import datetime

In [3]:
search_start = datetime(2019, 6, 19, 11, 0)
search_end = datetime(2019, 6, 19, 22, 0)

search_filter = {
    '$and': [
        {
            'sent': {
                '$gt': search_start,
            },
        },
        {
            'sent': {
                '$lt': search_end,
            },
        },
    ]
}

events = list(db.collection.find(search_filter))

In [4]:
len(events)

759

In [5]:
len([e for e in events if e['published']])

32

In [6]:
for e in events:
    e['published'] = False

In [7]:
def mock_by_ids(ids):
    return [event for event in events if event['id'] in ids]

def mock_publish(ids):
    for event in events:
        if event['id'] in ids:
            event['published'] = True

In [8]:
from unwetter.config import filter_event

def mock_update(new_events):
    filtered = []
    for event in new_events:
        if filter_event(event):
            if event['msg_type'] in ['Alert', 'Cancel']:
                filtered.append(event)

            elif not any(t['changed'] for t in event['has_changes']):
                continue

            elif any(t['changed'] for t in event['has_changes']):
                filtered.append(event)

            else:
                print(f'Event was not filtered 1: {event["id"]}')

        else:
            if event['msg_type'] in ['Alert', 'Cancel']:
                continue

            else:
                old_events = mock_by_ids(event['references'])

                if any((old_event['published'] and filter_event(old_event)) for old_event in old_events):
                    filtered.append(event)

                elif not any(old_event['published'] for old_event in old_events):
                    continue

                else:
                    print(f'Event was not filtered 2: {event["id"]}')

    mock_publish([event['id'] for event in filtered])
    return filtered

In [9]:
current_sent = events[0]['sent']

bins = []
current_bin = []

for event in events:
    if event['sent'] != current_sent:
        current_sent = event['sent']
        bins.append(current_bin)
        current_bin = []
    
    current_bin.append(event)

bins.append(current_bin)


In [10]:
sum(len(mock_update(bin)) for bin in bins)

Event was not filtered 2: 2.49.0.1.276.0.DWD.PVW.1560946920000.8deab210-1c6f-4d9e-ab2c-a2dd100bf69b.DEU
Event was not filtered 2: 2.49.0.1.276.0.DWD.PVW.1560948180000.a772f132-98eb-45c1-9570-a1203b030663.DEU
Event was not filtered 2: 2.49.0.1.276.0.DWD.PVW.1560949020000.d2f84e16-1192-42ee-b573-9e050b78bab3.DEU
Event was not filtered 2: 2.49.0.1.276.0.DWD.PVW.1560949320000.d4ac855a-686e-464d-a90f-78737deae44c.DEU
Event was not filtered 2: 2.49.0.1.276.0.DWD.PVW.1560949860000.ad53158c-51c2-4ec3-afca-22065a28a805.DEU
Event was not filtered 2: 2.49.0.1.276.0.DWD.PVW.1560964200000.4a0767cf-e5f4-490e-83ee-38e0f49aa956.DEU


32